<a href="https://colab.research.google.com/github/supergermy/colab_funfun/blob/main/af/examples/sequential_alanine_mutations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title install AlphaFold
import os
if not os.path.isdir("params"):
  print("installing AlphaFold...")
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  print("downloading AlphaFold params...")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar")
  os.system("tar -xf alphafold_params_2022-03-02.tar -C params")

installing AlphaFold...
downloading AlphaFold params...


In [ ]:
#@title af_random_alanine_mutation
import os
import random
import numpy as np
import tqdm.notebook
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
import tqdm.notebook
TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

#@markdown Specify sequence:
input_sequence = "AAELKTVISVESDPELSELIAEIARELGVDVVITRSALFEALRQVAADPAAAPAARALAEEAIAELRAVLRELLAKGYRVLLIDISVVPDELLADPTVRRLLAERLRWRAEMERELTEVLSNSLLQDGAVVLFKSFFSEEAARAFAAVLTEEAGFAAAILKVTGEPQSVTVTLTYNGVTATIEIVLLVGTHPPDVTVYLVISIDGADLPALAGKTLRVAVLVADVRSPKERLREAVERLLAALDAHPGVQLVVLLDTVAASAVRLRGGDREEVEARLREHLALLLELWRRVHARRAAAEE" #@param {type:"string"}
#num_mutations = 5 #@param {type:"raw"}

# Prepare the AlphaFold model
clear_mem()
af_model = mk_afdesign_model(protocol="hallucination")
af_model.prep_inputs(length=len(input_sequence))

# Create a directory for storing generated structures
output_dir = "random_sequential_mutations"
os.makedirs(output_dir, exist_ok=True)

# Generate structures for randomly and sequentially mutated sequences
seq_list = list(input_sequence)
num_mutations = len(input_sequence)
mutation_positions = random.sample(range(len(seq_list)), num_mutations)

with tqdm.notebook.tqdm(total=af_model._len, bar_format=TQDM_BAR_FORMAT) as pbar:
    for i, pos in enumerate(mutation_positions):
        # Mutate the current amino acid to alanine
        original_aa = seq_list[pos]
        seq_list[pos] = "A"
        mutated_seq = "".join(seq_list)
        
        # Predict the structure and save it
        af_model.set_seq(mutated_seq)
        af_model.predict(verbose=False, hard=False)
        af_model.save_current_pdb(f"{output_dir}/{i+1:04d}.pdb")
        af_model._save_results(verbose=False)

        pbar.update(1)
        
        ## Restore the original amino acid
        # seq_list[pos] = original_aa

# Create a movie of the generated structures
HTML(af_model.animate(dpi=100))


  0%|          | 0/300 [elapsed: 00:00 remaining: ?]